命名实体识别任务，先实现基于窗口的基线模型，然后进阶到RNN和GRU。中间利用对自动机的模拟和推导展示RNN的缺点，演示梯度剪裁的作用。

# 1 命名实体识别初步

定位命名实体并将其分类到：

    人名PER

    组织名ORG

    地名LOC

    其他MISC

加上非命名实体O一共5类。连续的标注视为同一个实体。比如样本$x^{(t)}$与标注$y^{(t)}$以及预测结果$\hat{y}^{(t)}$：
![3.jpg](3.jpg)

系统一共识别出3个命名实体，在token级别和entity级别各有评测方法。

token级别:

- P值为预测出的正确非O标签比上预测出的全部非O标签，于是$p=\frac{3}{4}$

- R值为预测出的正确非O标签比上正确答案的全部非O标签，于是$r=\frac{3}{6}$

- F1值是两者的调和平均：$F_1=\frac{2pr}{p+r}=\frac{6}{10}$

entity级别:

- P值为完美（不残缺不多余）识别的实体数量比上预测出的所有实体数量，于是$p=\frac{1}{3}$

- R值为完美识别的实体数量比上正确答案中的实体数量，于是$r=\frac{1}{3}$

- F1值是两者的调和平均：$F_1=\frac{2pr}{p+r}=\frac{1}{3}$

# 1 A window into NER

基线模型使用半径ww窗口中的特征$\boldsymbol{\tilde{x}}^{(t)}$预测$\boldsymbol{y}^{(t)}$：
![3-1-1.jpg](3-1-1.jpg)


模型为一个以ReLU为激活函数的隐藏层的神经网络，输出层为softmax，损失函数为交叉熵：

$$
\begin{align} 
    % 
    \boldsymbol{e}^{(t)} &= [ \boldsymbol{x}^{(t-w)}L, \dots, \boldsymbol{x}^{(t)}L, \dots, \boldsymbol{x}^{(t+w)}L ] \nonumber \\ 
    % 
    \boldsymbol{h}^{(t)} &= \text{ReLU}(\boldsymbol{e}^{(t)}W + \boldsymbol{b}_{1}) \nonumber \\ 
    % 
    \hat{\boldsymbol{y}}^{(t)} &= \text{softmax}(\boldsymbol{h}^{(t)}U + \boldsymbol{b}_{2}) \nonumber \\ 
    % 
    J &= \text{CE}(\boldsymbol{y}^{(t)}, \hat{\boldsymbol{y}}^{(t)}) \nonumber \\ 
    % 
    \text{CE}(\boldsymbol{y}^{(t)}, \hat{\boldsymbol{y}}^{(t)}) &= -\sum_{i} y_{i}^{(t)} \log(\hat{y}_{i}^{(t)})\text{ , } \nonumber 
\end{align}
$$


其中$L \in \mathbb{R}^{V \times D}$是词嵌入矩阵，$V$是词表大小，$D$是词嵌入维度，$\boldsymbol{h}^{(t)}$维度为$H$，即$H$是隐藏层维度，$\hat{\boldsymbol{y}}^{(t)}$的维度$C$，即$C$是分类数目，这里是5。

## 1-1  概念

请列举有歧义的命名实体？

方地/nr, 的/ude1, 茶/n, 喝/vg, 个/q, 一罐/mq

通常命名实体中含有低频词，为了泛化必须引入除了字符之外的特征，比如词性。这次作业为了简单，只使用字符特征。


## 1-2 维度和复杂度

如果窗口大小为$w$，则窗口特征$e^{(t)}$的维度是$(2w+1)×D$的行向量。$W$是$((2w+1)×D)×H$的矩阵。$U$是$H×C$的矩阵。

对于长度为$T$的句子来讲，计算复杂度是$O((2w+1)DHT)$，这是因为从输入到隐藏层是计算瓶颈。

## 1-3  实现基线模型

运行在`q1 window.py `中的`window-based classier`模型：

(1) 转换一个batch的输入，完善`make_windowed_data`函数。通过执行`python q1_window.py test1`命令来测试该函数的功能。

(2)在`WindowModel`类中完善`feed-forward`操作。通过运行如下命令`python q1_window.py test2`来测试。

(3)训练模型，`python q1_window.py train`。该命令耗时几分钟，其F1结果值应该>81%。模型的结果保存到`results/window/<timestamp>/`中，`<timestamp>`表示的迭代过程的中间结果。结果文件`results.txt`记录着模型在开发集上的校验结果。`log`文件则记录着打印的日志等，如混淆矩阵和F1的结果值信息。
最终可以通过如下命令进行模型交互：
`python q1_window.py shell -m results/window/<timestamp>/`

这里贴下predict方法，其他方法参加文件中的完整代码。

In [ ]:
def add_prediction_op(self):
    """Adds the 1-hidden-layer NN:
        h = Relu(xW + b1)
        h_drop = Dropout(h, dropout_rate)
        pred = h_dropU + b2
    Recall that we are not applying a softmax to pred. The softmax will instead be done in
    the add_loss_op function, which improves efficiency because we can use
    tf.nn.softmax_cross_entropy_with_logits
    When creating a new variable, use the tf.get_variable function
    because it lets us specify an initializer.
    Use tf.contrib.layers.xavier_initializer to initialize matrices.
    This is TensorFlow's implementation of the Xavier initialization
    trick we used in last assignment.
    Note: tf.nn.dropout takes the keep probability (1 - p_drop) as an argument.
        The keep probability should be set to the value of dropout_rate.
    Returns:
        pred: tf.Tensor of shape (batch_size, n_classes)
    """
    x = self.add_embedding()
    dropout_rate = self.dropout_placeholder
    ### YOUR CODE HERE (~10-20 lines)
    b1 = tf.get_variable(name='b1', shape = [self.config.hidden_size,], \
                         initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b2 = tf.get_variable(name='b2', shape = [self.config.n_classes], \
                         initializer=tf.contrib.layers.xavier_initializer(seed=2))
    W = tf.get_variable(name='W', shape = [self.config.n_window_features * self.config.embed_size, self.config.hidden_size], \
                        initializer=tf.contrib.layers.xavier_initializer(seed=3))
    U = tf.get_variable(name='U', shape = [self.config.hidden_size, self.config.n_classes], \
                        initializer=tf.contrib.layers.xavier_initializer(seed=4))
    z1 = tf.matmul(x,W) + b1
    h = tf.nn.relu(z1)
    h_drop = tf.nn.dropout(h, dropout_rate)
    pred = tf.matmul(h_drop,U) + b2
    ### END YOUR CODE
    return pred

##  1-4 分析结果:

测试`make_windowed_data`函数。`python q1_window.py test1`运行结果：

![3-1-3.jpg](3-1-3.jpg)

测试`WindowModel`模型，`python q1_window.py test2`运行结果：
注意：这个操作由于涉及到特定目录下的数据加载，所以要先切换到该程序所在的目录，再运行脚本。
![3-1-3_test2.JPG](3-1-3_test2.JPG)




1： entity-level 中最高的F1值是0.65,(这么低？？我怀疑是我哪里搞错了？？)。分析其对应的混淆矩阵可以看出，组织机构名分错的量还不少，表头中的go是指gold，gu是指guess的意思。所以，可以看出，将组织机构错分为人名是出错最多的地方。

2：由于窗口的限制，模型不擅长做完整连续的识别，如果增大窗口则会有所进步。

窗口=2，运行结果如下：

![3-1-3_test2_2.JPG](3-1-3_test2_2.JPG)

# 2 RNN

现在用RNN来解决这个问题。

![3-2-1.jpg](3-2-1.jpg)


$$
\begin{align} 
    % 
    \boldsymbol{e}^{(t)} &= \boldsymbol{x}^{(t)} L \nonumber \\ 
    % 
    \boldsymbol{h}^{(t)} &= \sigma \left( \boldsymbol{h}^{(t-1)} W_{h} + \boldsymbol{e}^{(t)} W_{x} + \boldsymbol{b}_{1} \right) \nonumber \\ 
    % 
    \hat{\boldsymbol{y}}^{(t)} &= \text{softmax} \left( \boldsymbol{h}^{(t)} U + \boldsymbol{b}_{2} \right) \text{ , } \nonumber 
    % 
\end{align}
$$

其中$L \in \mathbb{R}^{V \times D}$是词嵌入矩阵，$W_{h} \in \mathbb{R}^{H \times H}$，$W_{x} \in \mathbb{R}^{D \times H}$，和$\boldsymbol{b}_{1} \in \mathbb{R}^{H}$是RNN cell的参数。$U \in \mathbb{R}^{H \times C}$和$\boldsymbol{b}_{2} \in \mathbb{R}^{C}$是softmax的参数。$V$是词表大小，$D$是词嵌入维度，$H$是隐藏层神经元个数，$C$是分类的结果数量(本文是5)。

模型训练过程采用的交叉熵损失函数如下：

$$
\begin{align} 
 J &= \text{CE}(\boldsymbol{y}^{(t)}, \hat{\boldsymbol{y}}^{(t)}) \nonumber \\ 
    % 
    \text{CE}(\boldsymbol{y}^{(t)}, \hat{\boldsymbol{y}}^{(t)}) &= -\sum_{i} y_{i}^{(t)} \log(\hat{y}_{i}^{(t)})\text{ , } \nonumber 
\end{align}
$$
$$


## 2-1  模型参数增多了多少，计算复杂度又是多少？

RNN的参数比基线模型多多少
解答：

比基线模型多$(H \times H) - (2w \times D \times H)$个参数，因为多了一个隐藏状态转移矩阵$W_{h} \in \mathbb{R}^{H \times H}$，

而输入到隐藏单元的矩阵不再是$W \in \mathbb{R}^{(2w + 1) \times DH}$，而是$W_{x} \in \mathbb{R}^{D \times H}$。

计算复杂度：

还是输入到隐藏层、隐藏层到隐藏层的复杂度最高，总和为：

$$
O\left((D+H)HT\right)
$$


## 2-2 损失函数与F1


(1)举一个减小损失函数可能会降低$F1$的场景。

words/gold 对如下："The James/MISC scandal/MISC" (正确答案)

一种预测结果为"The James/MISC scandal/O",另一种预测结果为"The James/O scandal/O"。

第一种相对比第二种，交叉熵损失函数会更低，这是因为第一种预测多了一个token的结果值，即预测正确非O标签多了一个。但是同时，`entity-level F1`值也会下降，因为预测的实体是错误的。此时的精度会下降，但是召回率recall是保持不变的。


(2)为什么难以针对F1F1做优化呢？

$F1$ 不可导，需要在整个语料上进行计算，使其无法batch和并行化。

## 2-3  实现RNN单元

补充`q2_rnn_cell.py`中的`rnn_cell`函数，通过`python q2_rnn_cell.py test`对其进行测试。

运行结果：

![3-2-3-1.jpg](3-2-3-1.jpg)

## 2-4  RNN补零

实现RNN需要在整个句子上unroll，但句子的长度是不定的。一个解决方法是按最长的句子长度补零对齐。

假设最长句子长$M$ ，对长$T$的句子需要

1、为$x$和$y$补零向量，这些“零向量”依然是one-hot向量，代表NULL。

1、创建遮罩向量$\left( m^{(t)} \right)_{t=1}^{M}$，对所有$t≤T$是1，对所有$t>T$是0。

3、在补充了 $M−T$个token之后，损失和梯度都要做相应改动：

$$
\begin{align} 
    % 
    J &= \sum_{t=1}^{M} m^{(t)} \text{CE} \left( y^{(t)} , \hat{y}^{(t)} \right) \nonumber 
    % 
\end{align}
$$


(1)如果不改动损失函数，会产生什么结果？

如果不改的话，损失函数包含了预测为0的额外部分。这样的话，误差将来自补零的部分。此外影响参数学习。通过遮罩向量去掉它们可以解决这个问题。


(2)补充`q2 rnn.py`中的的`pad_sequences`函数。
补零的实现：

In [ ]:
def pad_sequences(data, max_length):
    """Ensures each input-output seqeunce pair in @data is of length
    @max_length by padding it with zeros and truncating the rest of the
    sequence.
    TODO: In the code below, for every sentence, labels pair in @data,
    (a) create a new sentence which appends zero feature vectors until
    the sentence is of length @max_length. If the sentence is longer
    than @max_length, simply truncate the sentence to be @max_length
    long.
    (b) create a new label sequence similarly.
    (c) create a _masking_ sequence that has a True wherever there was a
    token in the original sequence, and a False for every padded input.
    Example: for the (sentence, labels) pair: [[4,1], [6,0], [7,0]], [1,
    0, 0], and max_length = 5, we would construct
        - a new sentence: [[4,1], [6,0], [7,0], [0,0], [0,0]]
        - a new label seqeunce: [1, 0, 0, 4, 4], and
        - a masking seqeunce: [True, True, True, False, False].
    Args:
        data: is a list of (sentence, labels) tuples. @sentence is a list
            containing the words in the sentence and @label is a list of
            output labels. Each word is itself a list of
            @n_features features. For example, the sentence "Chris
            Manning is amazing" and labels "PER PER O O" would become
            ([[1,9], [2,9], [3,8], [4,8]], [1, 1, 4, 4]). Here "Chris"
            the word has been featurized as "[1, 9]", and "[1, 1, 4, 4]"
            is the list of labels. 
        max_length: the desired length for all input/output sequences.
    Returns:
        a new list of data points of the structure (sentence', labels', mask).
        Each of sentence', labels' and mask are of length @max_length.
        See the example above for more details.
    """
    ret = []
    # Use this zero vector when padding sequences.
    zero_vector = [0] * Config.n_features
    zero_label = 4 # corresponds to the 'O' tag
    for sentence, labels in data:
        ### YOUR CODE HERE (~4-6 lines)
        padN = max(max_length-len(sentence),0)
        sentence = sentence[0:max_length-padN]+[zero_vector]*padN
        labels = labels[0:max_length-padN]+ [zero_label]*(padN)
        mask = [True]*(max_length-padN) + [False]*padN
        ret.append((sentence,labels,mask))
        ### END YOUR CODE ###
    return ret

运行`python q2_rnn.py test1` 进行测试，测试结果：
![3-2-4-1.JPG](3-2-4-1.JPG)

## 2-5 实现RNN

运行`python q2_rnn.py test2` 进行测试，测试结果：

![3-2-5-1.JPG](3-2-5-1.JPG)


## 2-6 训练

运行`python q2_rnn.py train`进行训练。讲义上说CPU要跑两个小时，GPU要10-20分钟。要求结果$F1$值大于85%。

训练的模型会保存到`results/rnn/<timestamp>/`,其他与之前类似。

训练结果：
![3-2-6.jpg](3-2-6.jpg)

最终训练耗时也没有讲义上说的那么久。但是F1值却还没有达到85%，可能需要进一步调参吧。。。

可以通过：
`python q2_rnn.py shell -m results/rnn/<timestamp>/`与模型交互。


## 2-7 缺点：
这个朴素的RNN无法利用未来的特征来辅助当前决策，比如New York State University中的New容易被打上LOC标签，如果是biRNN则可能解决问题。另外模型也没有强制相邻token标签的连续性，solution中说引入 pair-wise agreements (i.e. using a CRF loss)可以解决这个问题。

# 3 GRU

课上讲过GRU可以有效降低梯度消失：

$$
\begin{align*} 
z_{t} &= \sigma(W^{(z)}x_{t} + U^{(z)}h_{t-1}+b_z)&~\text{(Update gate)}\\ 
r_{t} &= \sigma(W^{(r)}x_{t} + U^{(r)}h_{t-1}+b_r)&~\text{(Reset gate)}\\ 
\tilde{h}_{t} &= \operatorname{tanh}(r_{t}\circ  Uh_{t-1} + Wx_{t} +b_h)&~\text{(New memory)}\\ 
h_{t} &= (1 - z_{t}) \circ \tilde{h}_{t} + z_{t} \circ h_{t-1}&~\text{(Hidden state)} 
\end{align*}
$$

为了与GRU一致，将RNN记作:

$$
h_{t} = \sigma(r_{t}\circ  Uh_{t-1} + Wx_{t} +b_h)
$$

## 3-1 latch

Modeling latching behavior，就是首位是0或者1，随后的是n个连续的0。我们需要状态值$h$能够记住首个字符，而忽略后面的n个0。这种场景类似神经网络要学习如下的自动机。

用RNN模拟一个自动机，输出序列的第一个比特。
![3-3-1.jpg](3-3-1.jpg)

假设初始隐藏状态为0，激活函数替换为indicator函数：

$$
\sigma(x)\rightarrow 
\begin{cases} 
1,  & \text{if $x$ > 0} \\ 
0, & \text{otherwise} 
\end{cases}
$$

$$
\tanh(x)\rightarrow \begin{cases} 1,  & \text{if $x$ > 0} \\ 0, & \text{otherwise} \end{cases}
$$

(1)推导RNN实现自动机时候各参数需要满足的条件。

$$
h^{(t)} = \sigma(x^{(t)}U_h+h^{(t-1)}W_h+b_h)
$$


当$h^{(t-1)}=0,x^{(t)}=0$时，要让$h^{(t)}=0$则需要:

$$
\begin{align} 
\sigma(b_h)&=0\\ 
b_h&\leq 0 
\end{align}
$$


当$h^{(t-1)}=0,x^{(t)}=1$时，要让$h^{(t)}=1$则需要：
$$
\begin{align} \sigma(U_h+b_h)&=1\\ U_h+b_h&> 0 \end{align}
$$


当$h^{(t-1)}=1,x^{(t)}=0$时，要让$h^{(t)}=1$则需要:

$$
\begin{align} \sigma(W_h+b_h)&=1\\ W_h+b_h& > 0 \end{align}
$$

当$h^{(t-1)}=1,x^{(t)}=1$时，要让$h^{(t)}=1$则需要:

$$
\begin{align}
\sigma(W_h+U_h+b_h)&=1\\
W_h+U_h+b_h > 0
\end{align}
$$

也就是说必须满足:
$$
\begin{align} 
b_h&\leq0\\ 
U_h+b_h&>0\\ 
W_h+b_h& >0 
\end{align}
$$

(2) 当$w_r=u_r=b_r=b_z=b_h=0$,用GRU实现上述自动机

由于$w_r=u_r=b_r=b_z=b_h=0$，所以GRU单元简化为：

$$
\begin{align} 
z^{(t)}&=\sigma(x^{(t)}U_z+h^{(t-1)}W_z)\\ 
r^{(t)}&=0\\ 
\tilde{h}^{(t)}&=\tanh(x^{(t)}U_h)\\ 
h^{(t)}&=z^{(t)} \circ h^{(t-1)}+(1-z^{(t)}) \circ \tilde{h}^{(t)} 
\end{align}
$$


当$h^{(t-1)}=0,x^{(t)}=0$时，要让$h^{(t)}=0$则需要:
$$
\begin{align} 
z^{(t)}&=\sigma(0)=0 \\ 
\tilde{h}^{(t)}&=\tanh(0)=0 \\ 
h^{(t)}&=0 
\end{align}
$$

当$h^{(t-1)}=0,x^{(t)}=1$时，要让$h^{(t)}=1$则需要:

$$
\begin{align} 
z^{(t)}&=\sigma(U_z)\\ 
\tilde{h}^{(t)}&=\tanh(0)=0\\ 
h^{(t)}&=(1-\sigma(U_z)) \circ \tanh(U_h)=1\\ 
\rightarrow U_z &\leq 0 \\ 
U_h&>0 
\end{align}
$$

当$h^{(t-1)}=1,x^{(t)}=0$时，要让$h^{(t)}=1$则需要:

$$
\begin{align} 
z^{(t)}&=\sigma(W_z)\\ 
\tilde{h}^{(t)}&=\tanh(0)=0\\ 
h^{(t)}&=z^{(t)} \circ h^{(t-1)}=\sigma(W_z)=1\\ 
\rightarrow W_z&>0 
\end{align}
$$

当$h^{(t-1)}=1,x^{(t)}=1$时，要让$h^{(t)}=1$则需要:

$$
\begin{align} z^{(t)}&=\sigma(U_z+W_z)\\ \tilde{h}^{(t)}&=\tanh(U_h)=0\\ h^{(t)}&=z^{(t)}+(1-\sigma(U_z+W_z)) \circ \tanh(U_h)=1\\ \rightarrow U_z +W_z&>0 \end{align}
$$

综合起来：
$$
\begin{align} 
W_z&>0\\ 
U_z&\leq 0\\ 
U_h&>0 
\end{align}
$$

## 3-2 toggling(Modeling toggling behavior)

模拟开关，只要遇到1就切换状态：

![3-3-2.jpg](3-3-2.jpg)


(1)对于RNN：

当$x=0$时，RNN必须维持上一个状态不变：

$$
\begin{align} 
0\times w_h+0\times u_h+b_h &\leq0\\ 
1 \times w_h + 0 \times u_h + b_h &> 0\\ 
\rightarrow w_h&>0 
\end{align}
$$

当$x=1$时，RNN必须翻转上一个状态：

$$
\begin{align} 0\times w_h+0\times u_h+b_h &>0\\ 
1 \times w_h + 0 \times u_h + b_h &\leq 0 
\\ \rightarrow w_h&<0 \end{align}
$$

互相矛盾，所以RNN无法实现开关。

(2)对于GRU:

假设$w_r=u_r=b_z=b_h=0$。

先让$b_r=1$去关掉reset gate保持上一个状态。当$x=1$时，$u_z=1,b_z=w_z=0$，就有update gate为1。然后让$\tilde{h}$与$h$异号，有$u_h=0,w_h=−2$。

# 3-3 GRU实现
完善`q3_gru_cell.py`中的GRU单元，通过`python q3_gru_cell.py test` 对GRU cell进行测试。测试结果：

![3-3-3.jpg](3-3-3.jpg)

# 3-4 用TF内置的RNN模型学习latch

(1)
在`q3_gru.py`中完善`add_prediction_op`函数(使用TensorFlow中的动态RNN模型)

(2)完善`add_training_op`函数实现梯度归一化和梯度裁剪。

(3)测试程序：
```
python q3_gru.py predict -c [rnn|gru] [-g]
```
通过入参选择RNN还是GRU模型，参数$-g$是用以控制是否采用梯度裁剪。



## 3-5 结果对比分析

不带梯度裁剪和带梯度裁剪的RNN运行结果如下：

![q3-noclip-rnn.png](q3-noclip-rnn.png)

![q3-clip-rnn.png](q3-clip-rnn.png)

由于RNN梯度消失太快，还没达到最大梯度5就没了，所以裁剪不裁剪都无所谓。

而GRU时不时来个梯度大爆炸，还是很有用的：
![q3-noclip-gru.png](q3-noclip-gru.png)
![q3-clip-gru.png](q3-clip-gru.png)

可以设置`initFunc = tf.contrib.layers.xavier_initializer(uniform=False)`来进步观察这种梯度爆炸在GRU中的衰减情况。

## 3-6 GRU用于NER
采用GRU对之前的NER问题进行训练：

`python q2_rnn.py train -c gru`

训练结果：
![3-6.jpg](3-6.jpg)

训练耗时：8562s，好久。。。

代码中的彩蛋：

`python q3_gru.py dynamics`

会得到RNN和GRU的上一个隐藏状态和当前隐藏状态的变化图像。

但是由于机器上没有安装latex就没有再进一步尝试了。
